# Capstone Project     
## Machine Learning Engineer Nanodegree 
## New York City Taxi Trip Duration  
#### Wenhan Ji  
#### Dec. 10th, 2017

## Definition
### Project Overview

  Machine learning is a quite popular topic recent days. With more advanced algorithms being created and higher computing ability, we can train the machine learning model based on history data to predict feature with high accuracy. This kind of predictive model can be applied into diverse domains such as bio-informatics, economy, electrical engineering so as to optimize industrial products or help to make decisions.  
  

As to taxi duration prediction problem, some work can be found on the Internet.  
+ [Fare and Duration Prediction: A Study of New York City Taxi Rides](http://cs229.stanford.edu/proj2016/report/AntoniadesFadaviFobaAmonJuniorNewYorkCityCabPricing-report.pdf)    
In this work, the writer trained the lasso regression, random forest algorithm to predict NYC taxi duration.

+ [Using Metalearning for Prediction of Taxi Trip Duration Using Different Granularity Levels](https://link.springer.com/chapter/10.1007/978-3-319-24465-5_18)  
In this paper, the author propose the use of metalearning to address the problem of selecting the algorithm that generates the model with the most accurate predictions for each taxi.


  In my project, I trained a regression model using XGBoost algorithms to predicts the total ride duration of taxi trips in New York City. Once this model gets applied, the NYC taxi drivers can know how long the customers would take the taxi in advance with several features get detected.   

#### Dataset
  The primary dataset is one released by the NYC Taxi and Limousine Commission, which includes pickup time, geo-coordinates, number of passengers, and several other variables.   
  In addition, I used NYC 2016 weather dataset to add weather related features to each samples, including temperature, precipitation, snow fall and snow depth.

## Problem Statement
The goal is to create a regression model to predict NYC taxi trip duration; the tasks involved are the following:
1. Download 2016 NYC Yellow Cab trip record data and 2016 NYC weather data
2. Do the feature engineering work that create some new features based on existing features.
3. Visualize different features distribution to do the exploratory data analysis and data cleaning.
4. Define loss function and split training data and testing data.
5. Train XGBoost model and visualize the feature importance.

Metrics
Accuracy is Root Mean Squared Logarithmic Error.
<img style="width:40%" src="images/1.jpg" />
   
Where:  
ϵ is the RMSLE value (score)  
n is the total number of observations in the (public/private) data set  
pi is your prediction of trip duration  
ai is the actual trip duration for i  
log(x) is the natural logarithm of x  


**RMSLE** is used to compute the regression loss and it measures the ratio between actual and predicted.    
+ This measurement is useful when there is a wide range in the target variable, and you do not necessarily want to penalize large errors when the predicted and target values are themselves high. It is also effective when you care about percentage errors rather than the absolute value of errors.

## Analysis
### Data Exploration
##### nyc_taxi_trip_duration dataset    
11 features and 1458643 samples.   

<img style="width:50%" src="images/2.jpg" />  
+ id - a unique identifier for each trip
+ vendor_id - a code indicating the provider associated with the trip record
+ pickup_datetime - date and time when the meter was engaged
+ dropoff_datetime - date and time when the meter was disengaged
+ passenger_count - the number of passengers in the vehicle (driver entered value)
+ pickup_longitude - the longitude where the meter was engaged
+ pickup_latitude - the latitude where the meter was engaged
+ dropoff_longitude - the longitude where the meter was disengaged
+ dropoff_latitude - the latitude where the meter was disengaged
+ store_and_fwd_flag - This flag indicates whether the trip record was held in vehicle memory before sending to the vendor because the vehicle did not have a connection to the server - Y=store and forward; N=not a store and forward trip
+ trip_duration - duration of the trip in seconds  

<img style="width:90%" src="images/2_.jpg" /> 
From above the statistical computing result, some imformation can be extracted.
+ The vendor id has 2 category: 1 and 2.
+ Most of time, the passenger count is 1; the max passenger count is 9.
+ The max trip duration is exponentially higher then the 75 percentile one. This max amout of duration does not make any sense. So, there might exist some outliers.
  
##### 2016 nyc_weather dataset 
7 features and 366 samples.
 
<img style="width:50%" src="images/3.jpg" />

Weather data collected from the National Weather Service. It contains the first six months of 2016, for a weather station in central park. It contains for each day the minimum temperature, maximum temperature, average temperature, precipitation, new snow fall, and current snow depth. The temperature is measured in Fahrenheit and the depth is measured in inches. T means that there is a trace of precipitation.

<img style="width:90%" src="images/3_.jpg" />
+ In 2016 NYC, the max temperature is 96; the min temperature is -6; the average temperature is around 55.  
+ Most of days, there is no precipitation and snow.  

## Data Visualization (After Feature Engineering and Data Cleaning)
1. The below two plot visualize dropoff_clusters and pickup_clusters using scatter plot at corresponding location. These two features are computed by mini batch kmeans algorithm with n_clusters set to be 100. From the plots, we can find that this clustering makes sense with data distributes uniformly in each cluster.  
<img style="width:40%" src="images/4.jpg" />
<img style="width:40%" src="images/5.jpg" />

2. From the below histogram of trip duration, we will know that most trip duration is around 11 minutes.
<img style="width:60%" src="images/6.jpg" />  

3. From the below histogram of haversine_distance, we will know that most trip's haversine distance is around 2km.
<img style="width:60%" src="images/7.jpg" />  

4. From the below count plot of pickup hour, pickup weekday, pickup month, we will know that at around the hour of 18, people's need to take a taxi reached the peak. Maybe it's the time to get off work. The count of people take taxi at weekday of 4 is a little bit higher than other days. The count of people take taxi at month of 3 is a little bit higher than other weeks.
<img style="width:60%" src="images/8.jpg" /> 
<img style="width:60%" src="images/9.jpg" />  
<img style="width:60%" src="images/10.jpg" /> 

5. From the below count plot of vendor id, we can see that most of time, there's only one passenger that takes the taxi.
<img style="width:60%" src="images/11.jpg" />   

6. From the below count plot of vendor id, we can see the trip provider 1's records count is more than provider 2's.
<img style="width:30%" src="images/12.jpg" />  
 
7. From the below point plot of mean trip duration, we can get several observations.  
> + Trip records from vendor 1 has a higher mean trip duration then vendor 2's records.  
> + At weekday of 3, people's taxi mean trip duration is the highest. At weekday of 6, people's taxi mean trip duration is the lowest.    
> + At around the hour of 16, people's taxi mean trip duration is the highest. At hour of 6, people's taxi mean trip duration is the lowest.    

<img style="width:60%" src="images/13.jpg" />
<img style="width:60%" src="images/14.jpg" />

## Algorithms and Techniques
1. Mini Batch KMeans   
The machine learning algorithm that cluster dropoff longitude, dropoff latitude, pickup longitude, pickup latitude features is Mini Batch KMeans. In this way, I created pickup_cluster and dropoff_cluster two new features. This algorithm can be seen below.
<img style="width:60%" src="images/15.jpg" /> 
In cluster analysis, the Mini Batch KMeans algorithm can be used to partition the input data set into k partitions.   
The Mini Batch KMeans contains 2 steps:
> 1. Data assigment step:  
Each centroid defines one of the clusters. In this step, each data point is assigned to its nearest centroid, based on the squared Euclidean distance. 
> 2. Centroid update step:  
In this step, the centroids are recomputed. This is done by taking the mean of all data points assigned to that centroid's cluster.  

2. Haversine Distance  
Also, with dropoff longitude, dropoff latitude, pickup longitude, pickup latitude features, I computed haversine distance. The formula can be seen below.
<img style="width:60%" src="images/16.jpg" />   

3. XGBoost  
The predictive regression algorithm that train the feature data so as to predict trip duration is XGBoost.
XGBoost is an open-source software library which provides the gradient boosting framework for C++, Java, Python, R, and Julia. It works on Linux, Windows, and macOS. From the project description, it aims to provide a "Scalable, Portable and Distributed Gradient Boosting (GBM, GBRT, GBDT) Library".
XGBoost is short for **Extreme Gradient Boosting**, where the term “Gradient Boosting” is proposed in the paper Greedy Function Approximation: A Gradient Boosting Machine, by Friedman. XGBoost is based on this original model. This is a tutorial on gradient boosted trees, and most of the content is based on these slides by the author of xgboost.    
>**Gradient boosting** is a machine learning technique for regression and classification problems, which produces a prediction model in the form of an ensemble of weak prediction models, typically decision trees. It builds the model in a stage-wise fashion like other boosting methods do, and it generalizes them by allowing optimization of an arbitrary differentiable loss function.  
In GBDT the trees are mostly trained sequentially so that each subsequent tree trains on examples that are poorly labelled by the previously fitted trees.    

>There are three key advantages of GBDTs:
>1. Better results with smaller data: DNNs work well for large datasets, but they are so overparametrized compared to GBDTs that GBDTs are likely to outperform DNNs in the 100–10,000 range of training instances.  
>2. Minimal fine tuning: GBDTs can be blindly pointed at a problem and you can expect sane results.  
>3. Super fast training: Training a GBDT can be done very rapidly; usually seconds or minutes, hours at most.

#### Benchmark model
1. Linear Regression  
>+ Linear regression is a linear approach for modeling the relationship between a scalar dependent variable y and one or more explanatory variables (or independent variables) denoted X.    
>+ In this project, the relationship between trip duration and other features cannot be linear. So this model will be underfitting and the loss will be a little bit high.  

2. K Neighbors Regressor with n_neighbors=5
>+ k-nearest neighbors algorithm (k-NN) is a non-parametric method used for classification and regression. In both cases, the input consists of the k closest training examples in the feature space. The output depends on whether k-NN is used for classification or regression:
>+ In k-NN regression, the output is the property value for the object. This value is the average of the values of its k nearest neighbors.    
>+ k-NN is a type of instance-based learning, or lazy learning, where the function is only approximated locally and all computation is deferred until classification. The k-NN algorithm is among the simplest of all machine learning algorithms.
>+ In this project, KNN will also be underfitting due to the simplicity of model. The result will have high bias, so the loss will be a little bit high. 

3. Random Forest Regressor with n_estimators=100  
>+ Random forests or random decision forests are an ensemble learning method for classification, regression and other tasks, that operate by constructing a multitude of decision trees at training time and outputting the class that is the mode of the classes (classification) or mean prediction (regression) of the individual trees.   
>+ Random decision forests correct for decision trees' habit of overfitting to their training set.  
>+ The training algorithm for random forests applies the general technique of bootstrap aggregating, or bagging, to tree learners. Recall that the key to bagging is that trees are repeatedly fit to bootstrapped subsets of the observations. One can show that on average, each bagged tree makes use of around two-thirds of the observations.
>+ But when building these decision trees,each time a split in a tree is considered, a random sample of m predictors is chosen as split candidates from the full set of p predictors. Using a small value of m in building a random forest will typically be helpful when we have a large number of correlated predictors.    
>+ This bootstrapping procedure leads to better model performance because it decreases the variance of the model, without increasing the bias .

## Methodology
#### Data Preprocessing
The preprocessing done in the 'Feature Engineering' and 'Data Cleaning' part of notebook consists of the following steps:
1. Create pickup_cluster feature using Mini Batch KMeans algorithms with the help of 4 location features: pickup_latitude, pickup_longitude, dropoff_latitude, dropoff_longitude.
2. Compute haversine_distance feature using haversine package with pickup and dropoff location information.
3. Parse out date, weekday, hour, month, day feature from pickup_datetime feature.
4. Merge NYC taxi duration kaggle dataset and 2016 NYC whether dataset based on the date feature.
5. Encode the categorical feature store_and_fwd_flag and vendor_id to 0,1.
6. Clean datasets based on the visualization plots and computed max, min, mean parameters of each feature. I deleted records that trip duration is less than 1 minutes and over 12 hours, trip haversine distance less than 0.01km and over 400km. These deleted records do not make sense, as the trip is too short or too long.
7. Split the training and testing data and test set size is 20% of original dataset.


## Implementation
#### Metrics Define
The rmsle_eval_sklearn function is used to evaluate the benchmark model performance.  
The rmsle_eval function is used to evaluate the XGBoost model performance.  
<img style="width:60%" src="images/rmsle.jpg" />  
#### Train Test Split
Set 20% of original dataset as test part.
<img style="width:80%" src="images/train_test_split.jpg" /> 
#### Benchmark model from sklearn:
1. Linear Regression model  
<img style="width:80%" src="images/lr.jpg" />  
2. K Neighbors Regressor with n_neighbors=5
<img style="width:80%" src="images/knn.jpg" />  
3. Random Forest Regressor with n_estimators=10
<img style="width:80%" src="images/rf.jpg" />  

#### XGBoost model:
First we have to define the parameters of XGBoost:
[Here is the tutorial for parameters tuning](https://github.com/dmlc/xgboost/blob/master/doc/parameter.md)
<img style="width:30%" src="images/xgb_params.jpg" />  
>+ **eta** [default=0.3, alias: learning_rate]  
step size shrinkage used in update to prevents overfitting. After each boosting step, we can directly get the weights of new features. and eta actually shrinks the feature weights to make the boosting process more conservative.
>+ **min_child_weight** [default=1]  
minimum sum of instance weight (hessian) needed in a child. If the tree partition step results in a leaf node with the sum of instance weight less than min_child_weight, then the building process will give up further partitioning. 
>+ **subsample**:    
subsample ratio of the training instance. Setting it to 0.5 means that XGBoost randomly collected half of the data instances to grow trees and this will prevent overfitting.
>+ **colsample_bytree** [default=1]    
subsample ratio of columns when constructing each tree.
>+ **max_depth** [default=6]  
maximum depth of a tree, increase this value will make the model more complex / likely to be overfitting. 0 indicates no limit, limit is required for depth-wise grow policy.

Then to set the training and testing data into DMetrix format needed by XGBoost package.
<img style="width:50%" src="images/xgb_dmetrix.jpg" /> 
Finally, we can train the XGBoost model.

<img style="width:50%" src="images/xgb.jpg" />
>+ **evals_result** use the root mean squared log error we defined above
>+ **early stopping** will monitor test-rmsle, the model will train until test-rmsle hasn't improved in 10 rounds.
>+ **maximize** defines whether to maximize feval.

## Results-Model Evaluation 
#### Benchmark model from sklearn:
1. linear model: Root Mean Squared Logarithmic Error= 0.074
2. K Neighbors Regressor: Root Mean Squared Logarithmic Error= 0.073
3. Random Forest Regressor: Root Mean Squared Logarithmic Error= 0.047

>The benchmark model's loss on test set makes sense. The linear model and KNN's error is high because it suffer from the bias problem. The models are too simple that it can not fit the training data well.    
>The random forest model performs better than previous 2 as the bootstrapping procedure decreases the variance of the model. 

#### XGBoost model:
Root Mean Squared Logarithmic Error= 0.044  
>The prediction error on test set is lower than all benchmark model thanks to the good generalization ability of xgboost.  

The graph below shows the traing and testing loss seperately with training rounds increase.   
We can figure out the after 100 rounds, the error decrease very slowly.  
The traing error decreases faster than testing error. As the gap is small, so the overfitting problem is small.   
And it can also be concluded that this model generalize well due to the small prediction loss on test set. So the model can generalize well.
<img style="width:50%" src="images/results.jpg" />


## Conclusion
#### Reflection
The process used for this project can be summarized using the following steps:
1. An initial problem and relevant, public datasets were found
2. The data was downloaded 
3. Compute and create new features based on existing features
4. Visualize features to do the exploratory data analysis and data cleaning
5. Train test split and the test data function as the future unknown data.
6. Benchmark model was created for the regressor
7. The regressor was trained using the data
8. Do the model evaluation based on test set and loss function.

I found steps 3 the most difficult. The first time I faced the dropoff and pickup location data, I have no idea how these features can help me improve my model. It's after I read some documents then know they can help to compute the haversine distance. And the KMeans clustering can be applied to cluster the location. 

The visualization part is also import that the plots help me have a deeper insight of different features distribution so that more information can be extracted.

Compared with benchmark model, XGBoost model show a better performance to do this regression task


#### Potential solutions to improve the model
Find some new datasets related to 2016 NYC. With more features considered, the model may perform better.
